In [ ]:
# Step 1: Install dependencies
!pip install pandas numpy lightfm scipy

# Step 2: Load and preprocess dataset
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

# Load CSV (make sure you've uploaded it or mounted Drive)
df = pd.read_csv(
    "Assignment-1_Data.csv",
    sep=';',              # semicolon separator
    decimal=',',          # comma decimal separator for Price
    parse_dates=['Date'], # parse date column
    dayfirst=True         # day before month in date format
)
print(f"Loaded {len(df)} records")
df = df.dropna(subset=['CustomerID', 'Itemname'])

# Map CustomerID and Itemname to categorical codes for LightFM
df['user_id'] = df['CustomerID'].astype('category').cat.codes
df['item_id'] = df['Itemname'].astype('category').cat.codes

# Step 3: Create interaction matrix
interactions = coo_matrix(
    (df['Quantity'], (df['user_id'], df['item_id']))
)
print(f"Interaction matrix shape: {interactions.shape}")

# Step 4: Train LightFM model
from lightfm import LightFM

model = LightFM(loss='warp')
model.fit(interactions, epochs=10, num_threads=4)

# Step 5: Build mappings for user and item IDs
user_map = dict(zip(df['CustomerID'], df['user_id']))
item_map = dict(zip(df['Itemname'], df['item_id']))

# Step 6: Define recommendation function
def recommend_for_user(model, real_user_id, user_map, item_map, num_recommendations=5):
    if real_user_id not in user_map:
        print(f"User ID {real_user_id} not found.")
        return
    
    user_internal_id = user_map[real_user_id]
    n_items = len(item_map)
    
    scores = model.predict(user_internal_id, np.arange(n_items))
    top_items = np.argsort(-scores)[:num_recommendations]
    
    inv_item_map = {v: k for k, v in item_map.items()}
    recommended_items = [inv_item_map[i] for i in top_items]
    
    print(f"Top {num_recommendations} recommendations for User {real_user_id}:")
    for i, item in enumerate(recommended_items, 1):
        print(f"{i}. {item}")

# Step 7: Example usage
sample_user = list(user_map.keys())[0]
recommend_for_user(model, sample_user, user_map, item_map)

# Step 8: Save model and maps
import pickle

with open('lightfm_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('user_map.pkl', 'wb') as f:
    pickle.dump(user_map, f)

with open('item_map.pkl', 'wb') as f:
    pickle.dump(item_map, f)

print("Model and mappings saved!")
